In [1]:
import pandas as pd
import numpy as np
import datetime
from sklearn import preprocessing
from keras import layers
from keras.models import Sequential
from keras.utils import plot_model
import matplotlib.pyplot as plt
import sys
sys.path.append(r'D:\jupyter files\waiting_time_project\my_tools')
import tools_for_os.for_df as ml_df
import tools_for_os.for_file as ml_fl
import warnings
warnings.filterwarnings("ignore")

Using TensorFlow backend.


In [2]:
data_path = 'D:\\jupyter files\\data_waiting_time_project\\preprocess_data\\'

pat_duration_LSTM_data_path = data_path + 'patient_duration_LSTM_data\\'
ml_fl.create_folder(pat_duration_LSTM_data_path)

D:\jupyter files\data_waiting_time_project\preprocess_data\patient_duration_LSTM_data\ already existed!


In [3]:
data_part1 = pd.read_csv(data_path + 'data_part1.csv', index_col = 0)
data_part2 = pd.read_csv(data_path + 'data_part2.csv', index_col = 0)

In [4]:
# 把字符串转成datetime
def str_to_Datetime(st):
    dt = datetime.datetime.strptime(st, "%Y-%m-%d %H:%M:%S")
    return dt

In [5]:
feature_num = ['Scheduled_duration', 'Actual_duration',
               'age', 'TreatmentTime_total', 'ImagesTaken_total',
               'MU_total', 'MUCoeff_total', 'Interval_scheduled']

# RadiationId
feature_cate = ['dxt_AliasName', 'Sex', 'AliasSerNum',
                'month', 'week', 'hour', 'DoctorSerNum', 
                'TreatmentOrientation', 'FractionNumber',
                'UserName', 'CourseId', 'ResourceSerNum']

In [6]:
# 因为需要进行onehot encoding，所以在拼接数据之前，先进行数据格式的处理
for col in feature_cate:
    try:
        data_part1[col].fillna('Unknown', inplace = True)
        data_part1[col] = data_part1[col].astype(str)
    except:
        pass
    
    try:
        data_part2[col].fillna('Unknown', inplace = True)
        data_part2[col] = data_part2[col].astype(str)
    except:
        pass

for col in feature_num:
    try:
        data_part1.fillna(0, inplace = True)
    except:
        pass
    try:
        data_part2.fillna(0, inplace = True)
    except:
        pass
    
# data_num = log1p(data_num)

In [7]:
# one hot encoder
label_encoder_dxt_AliasName = preprocessing.OneHotEncoder(handle_unknown = 'ignore', sparse = False)
label_encoder_dxt_AliasName.fit(data_part1.dxt_AliasName.values.reshape(-1, 1))

label_encoder_Sex = preprocessing.OneHotEncoder(handle_unknown = 'ignore', sparse = False)
label_encoder_Sex.fit(data_part1.Sex.values.reshape(-1, 1))

label_encoder_AliasSerNum = preprocessing.OneHotEncoder(handle_unknown = 'ignore', sparse = False)
label_encoder_AliasSerNum.fit(data_part1.AliasSerNum.values.reshape(-1, 1))

label_encoder_month = preprocessing.OneHotEncoder(handle_unknown = 'ignore', sparse = False)
label_encoder_month.fit(data_part1.month.values.reshape(-1, 1))

label_encoder_week = preprocessing.OneHotEncoder(handle_unknown = 'ignore', sparse = False)
label_encoder_week.fit(data_part1.week.values.reshape(-1, 1))

label_encoder_hour = preprocessing.OneHotEncoder(handle_unknown = 'ignore', sparse = False)
label_encoder_hour.fit(data_part1.hour.values.reshape(-1, 1))

label_encoder_DoctorSerNum = preprocessing.OneHotEncoder(handle_unknown = 'ignore', sparse = False)
label_encoder_DoctorSerNum.fit(data_part1.DoctorSerNum.values.reshape(-1, 1))

label_encoder_TreatmentOrientation = preprocessing.OneHotEncoder(handle_unknown = 'ignore', sparse = False)
label_encoder_TreatmentOrientation.fit(data_part1.TreatmentOrientation.values.reshape(-1, 1))

label_encoder_FractionNumber = preprocessing.OneHotEncoder(handle_unknown = 'ignore', sparse = False)
label_encoder_FractionNumber.fit(data_part2.FractionNumber.values.reshape(-1, 1))

label_encoder_UserName = preprocessing.OneHotEncoder(handle_unknown = 'ignore', sparse = False)
label_encoder_UserName.fit(data_part2.UserName.values.reshape(-1, 1))

label_encoder_CourseId = preprocessing.OneHotEncoder(handle_unknown = 'ignore', sparse = False)
label_encoder_CourseId.fit(data_part2.CourseId.values.reshape(-1, 1))

label_encoder_ResourceSerNum = preprocessing.OneHotEncoder(handle_unknown = 'ignore', sparse = False)
label_encoder_ResourceSerNum.fit(data_part2.ResourceSerNum.values.reshape(-1, 1))

OneHotEncoder(categorical_features=None, categories=None, drop=None,
              dtype=<class 'numpy.float64'>, handle_unknown='ignore',
              n_values=None, sparse=False)

==============================================================

#### categorical feature 不同取值的个数

In [8]:
# feature_num = ['Scheduled_duration', 'Actual_duration', 'Actual_duration',
#                'age', 'TreatmentTime_total', 'ImagesTaken_total',
#                'MU_total', 'MUCoeff_total']

# # RadiationId
# feature_cate = ['dxt_AliasName', 'Sex', 'AliasSerNum',
#                 'month', 'week', 'hour', 'DoctorSerNum', 
#                 'TreatmentOrientation', 'FractionNumber',
#                 'UserName', 'CourseId', 'ResourceSerNum']

# feature_count1 = pd.DataFrame({})
# for col in feature_cate:
#     try:
#         n = len(data_part1[col].unique())
#         feature_count1[col] = [n]
#     except:
#         pass
    
# feature_count2 = pd.DataFrame({})
# for col in feature_cate:
#     try:
#         n = len(data_part2[col].unique())
#         feature_count2[col] = [n]
#     except:
#         pass

==============================================================

In [9]:
data_part1_grouped = data_part1.groupby('AppointmentSerNum')
count_appt = data_part1_grouped.count()
appt_one_list = count_appt[count_appt.Sex == 1].index.tolist()
appt_more_list = count_appt[count_appt.Sex > 1].index.tolist()

print(f'{len(appt_one_list)} with one appointment')
print(f'{len(appt_more_list)} with more appointment')

69142 with one appointment
251892 with more appointment


In [18]:
def get_list(x):
    x = list(x)
    if len(x) == 1:
        return x[0]
    else:
        return x

In [43]:
new_appt = pd.DataFrame({})

print('Start cateorical features')
print('\nStart PatientSerNum')
new_appt['PatientSerNum'] = data_part1.groupby('AppointmentSerNum').PatientSerNum.apply(set)
new_appt['PatientSerNum'] = new_appt['PatientSerNum'].apply(lambda x: get_list(x))

print('Start Sex')
new_appt['Sex'] = data_part1.groupby('AppointmentSerNum').Sex.apply(set)
new_appt['Sex'] = new_appt['Sex'].apply(lambda x: get_list(x))

print('Start DoctorSerNum')
new_appt['DoctorSerNum'] = data_part1.groupby('AppointmentSerNum').DoctorSerNum.apply(set)
new_appt['DoctorSerNum'] = new_appt['DoctorSerNum'].apply(lambda x: get_list(x))

print('Start date')
new_appt['date'] = data_part1.groupby('AppointmentSerNum').date.apply(set)
new_appt['date'] = new_appt['date'].apply(lambda x: get_list(x))

print('Start ScheduledStartTime')
new_appt['ScheduledStartTime'] = data_part1.groupby('AppointmentSerNum').ScheduledStartTime.apply(set)
new_appt['ScheduledStartTime'] = new_appt['ScheduledStartTime'].apply(lambda x: get_list(x))

print('Start ScheduledEndTime')
new_appt['ScheduledEndTime'] = data_part1.groupby('AppointmentSerNum').ScheduledEndTime.apply(set)
new_appt['ScheduledEndTime'] = new_appt['ScheduledEndTime'].apply(lambda x: get_list(x))

print('Start ActualStartDate')
new_appt['ActualStartDate'] = data_part1.groupby('AppointmentSerNum').ActualStartDate.apply(set)
new_appt['ActualStartDate'] = new_appt['ActualStartDate'].apply(lambda x: get_list(x))

print('Start ActualEndDate')
new_appt['ActualEndDate'] = data_part1.groupby('AppointmentSerNum').ActualEndDate.apply(set)
new_appt['ActualEndDate'] = new_appt['ActualEndDate'].apply(lambda x: get_list(x))

print('Start dxt_AliasName')
new_appt['dxt_AliasName'] = data_part1.groupby('AppointmentSerNum').dxt_AliasName.apply(set)
new_appt['dxt_AliasName'] = new_appt['dxt_AliasName'].apply(lambda x: get_list(x))

print('Start AliasSerNum')
new_appt['AliasSerNum'] = data_part1.groupby('AppointmentSerNum').AliasSerNum.apply(set)
new_appt['AliasSerNum'] = new_appt['AliasSerNum'].apply(lambda x: get_list(x))

print('Start CourseSerNum')
new_appt['CourseSerNum'] = data_part1.groupby('AppointmentSerNum').CourseSerNum.apply(set)
new_appt['CourseSerNum'] = new_appt['CourseSerNum'].apply(lambda x: get_list(x))

print('Start PlanSerNum')
new_appt['PlanSerNum'] = data_part1.groupby('AppointmentSerNum').PlanSerNum.apply(set)
new_appt['PlanSerNum'] = new_appt['PlanSerNum'].apply(lambda x: get_list(x))

print('Start TreatmentOrientation')
new_appt['TreatmentOrientation'] = data_part1.groupby('AppointmentSerNum').TreatmentOrientation.apply(set)
new_appt['TreatmentOrientation'] = new_appt['TreatmentOrientation'].apply(lambda x: get_list(x))

print('Start month')
new_appt['month'] = data_part1.groupby('AppointmentSerNum').month.apply(set)
new_appt['month'] = new_appt['month'].apply(lambda x: get_list(x))

print('Start week')
new_appt['week'] = data_part1.groupby('AppointmentSerNum').week.apply(set)
new_appt['week'] = new_appt['week'].apply(lambda x: get_list(x))

print('Start hour')
new_appt['hour'] = data_part1.groupby('AppointmentSerNum').hour.apply(set)
new_appt['hour'] = new_appt['hour'].apply(lambda x: get_list(x))

print('Start AppointmentSerNum')
new_appt['AppointmentSerNum'] = new_appt.index.tolist()


print('Start numberical features')
print('\nStart age')
new_appt['age'] = data_part1.groupby('AppointmentSerNum').age.mean()

print('Start Scheduled_duration')
new_appt['Scheduled_duration'] = data_part1.groupby('AppointmentSerNum').Scheduled_duration.mean()

print('Start Actual_duration')
new_appt['Actual_duration'] = data_part1.groupby('AppointmentSerNum').Actual_duration.mean()

Start cateorical features

Start PatientSerNum
Start Sex
Start DoctorSerNum
Start date
Start ScheduledStartTime
Start ScheduledEndTime
Start ActualStartDate
Start ActualEndDate
Start dxt_AliasName
Start AliasSerNum
Start CourseSerNum
Start PlanSerNum
Start TreatmentOrientation
Start month
Start week
Start hour
Start AppointmentSerNum
Start numberical features

Start age
Start Scheduled_duration
Start Actual_duration


In [61]:
new_appt_ = new_appt.copy()

In [45]:
# data_part1_grouped = data_part1.groupby('AppointmentSerNum')
# count_appt = data_part1_grouped.count()
# appt_one_list = count_appt[count_appt.Sex == 1].index.tolist()
# appt_more_list = count_appt[count_appt.Sex > 1].index.tolist()

# print(f'{len(appt_one_list)} with one appointment')
# print(f'{len(appt_more_list)} with more appointment')


# # key_list = list(data_part1_grouped.groups.keys())
# data_part1_new = pd.DataFrame({})
# for i in range(len(appt_more_list)):
#     if (i + 1) % 500 == 0:
#         print(f'\n{i+1} appointment data')
        
#     sample = data_part1_grouped.get_group(appt_more_list[i])
#     new_appt = pd.DataFrame({
#         'PatientSerNum': sample.PatientSerNum.tolist()[0],
#         'AppointmentSerNum': sample.AppointmentSerNum.tolist()[0],
#         'Sex': sample.Sex.tolist()[0],
#         'age': sample.age.tolist()[0],
#         'DoctorSerNum': sample.DoctorSerNum.tolist()[0],
#         'month': sample.month.tolist()[0],
#         'date': sample.date.tolist()[0],
#         'week': sample.week.tolist()[0],
#         'hour': sample.hour.tolist()[0],
#         'Scheduled_duration': sample.Scheduled_duration.tolist()[0],
#         'Actual_duration': sample.Actual_duration.tolist()[0],
#         'ScheduledStartTime': sample.ScheduledStartTime.tolist()[0],
#         'ScheduledEndTime': sample.ScheduledEndTime.tolist()[0],
#         'ActualStartDate': sample.ActualStartDate.tolist()[0],
#         'ActualEndDate': sample.ActualEndDate.tolist()[0],
        
#         'dxt_AliasName': [list(set(sample.dxt_AliasName.tolist()))], 
#         'AliasSerNum': [list(set(sample.AliasSerNum.tolist()))], 
#         'CourseSerNum': [list(set(sample.CourseSerNum.tolist()))], 
#         'PlanSerNum': [list(set(sample.PlanSerNum.tolist()))], 
#         'TreatmentOrientation': [list(set(sample.TreatmentOrientation.tolist()))],
        
#     })
    
#     data_part1_new = pd.concat([data_part1_new, new_appt], axis = 0)
# data_part1_new.sort_values(by = ['PatientSerNum', 'AppointmentSerNum', 'ScheduledStartTime'], inplace = True)
# data_part1_new.reset_index(drop = True, inplace = True)

In [59]:
new_treat = pd.DataFrame({})

print('Start categorical features')
print('\nStart FractionNumber')
new_treat['FractionNumber'] = data_part2.groupby(['PatientSerNum', 'date']).PatientSerNum.apply(set)
new_treat['FractionNumber'] = new_treat['FractionNumber'].apply(lambda x: get_list(x))

print('Start UserName')
new_treat['UserName'] = data_part2.groupby(['PatientSerNum', 'date']).UserName.apply(set)
new_treat['UserName'] = new_treat['UserName'].apply(lambda x: get_list(x))

print('Start RadiationSerNum')
new_treat['RadiationSerNum'] = data_part2.groupby(['PatientSerNum', 'date']).RadiationSerNum.apply(set)
new_treat['RadiationSerNum'] = new_treat['RadiationSerNum'].apply(lambda x: get_list(x))

print('Start RadiationId')
new_treat['RadiationId'] = data_part2.groupby(['PatientSerNum', 'date']).RadiationId.apply(set)
new_treat['RadiationId'] = new_treat['RadiationId'].apply(lambda x: get_list(x))

print('Start ResourceSerNum')
new_treat['ResourceSerNum'] = data_part2.groupby(['PatientSerNum', 'date']).ResourceSerNum.apply(set)
new_treat['ResourceSerNum'] = new_treat['ResourceSerNum'].apply(lambda x: get_list(x))

print('Start CourseId')
new_treat['CourseId'] = data_part2.groupby(['PatientSerNum', 'date']).CourseId.apply(set)
new_treat['CourseId'] = new_treat['CourseId'].apply(lambda x: get_list(x))

print('Start PatientSerNum')
new_treat['PatientSerNum'] = new_treat.index.get_level_values(level = 0).tolist()

print('Start date')
new_treat['date'] = new_treat.index.get_level_values(level = 1).tolist()


print('Start numberical features')
print('\nStart ImagesTaken_total')
new_treat['ImagesTaken_total'] = data_part2.groupby(['PatientSerNum', 'date']).ImagesTaken.sum()

print('Start MU_total')
new_treat['MU_total'] = data_part2.groupby(['PatientSerNum', 'date']).MU.sum()

print('Start MUCoeff_total')
new_treat['MUCoeff_total'] = data_part2.groupby(['PatientSerNum', 'date']).MUCoeff.sum()

print('Start TreatmentTime_total')
new_treat['TreatmentTime_total'] = data_part2.groupby(['PatientSerNum', 'date']).TreatmentTime.sum()

Start categorical features

Start FractionNumber
Start UserName
Start RadiationSerNum
Start RadiationId
Start ResourceSerNum
Start CourseId
Start PatientSerNum
Start date
Start numberical features

Start ImagesTaken_total
Start MU_total
Start MUCoeff_total
Start TreatmentTime_total


In [62]:
new_treat_ = new_treat.copy()

In [64]:
new_appt_.reset_index(drop = True, inplace = True)

new_treat_.reset_index(drop = True, inplace = True)

In [65]:
# data_part2_grouped = data_part2.groupby(['PatientSerNum', 'date'])
# count_pat_appt = data_part2_grouped.count()
# treat_one_list = count_pat_appt[count_pat_appt.RadiationHstryAriaSer == 1].index.tolist()
# treat_more_list = count_pat_appt[count_pat_appt.RadiationHstryAriaSer > 1].index.tolist()

# print(f'{len(treat_one_list)} with one treatment')
# print(f'{len(treat_more_list)} with more treatment')


# # key_list = list(data_part2_grouped.groups.keys())
# data_part2_new = pd.DataFrame({})
# for i in range(len(treat_more_list)):
#     if (i+1) % 500 == 0:
#         print(f'\n{i+1} treatment data')
        
#     sample = data_part2_grouped.get_group(treat_more_list[i])
#     try:
#         new_treat = pd.DataFrame({
#             'PatientSerNum': sample.PatientSerNum.tolist()[0],
#             'FractionNumber': sample.FractionNumber.tolist()[0],
#             'date': sample.date.tolist()[0],

#             'UserName': [list(set(sample.UserName.tolist()))], 
#             'RadiationSerNum': [list(set(sample.RadiationSerNum.tolist()))], 
#             'RadiationId': [list(set(sample.RadiationId.tolist()))], 
#             'ResourceSerNum': [list(set(sample.ResourceSerNum.tolist()))], 
#             'CourseId': [list(set(sample.CourseId.tolist()))],

#             'ImagesTaken_total': sum(sample.ImagesTaken.tolist()),
#             'MU_total': sum(sample.MU.tolist()),
#             'MUCoeff_total': sum(sample.MUCoeff.tolist()),
#             'TreatmentTime_total': sum(sample.TreatmentTime.tolist()),
#         })
#         data_part2_new = pd.concat([data_part2_new, new_treat], axis = 0)
#     except:
#         print(treat_more_list[i])
    
    
# data_part2_new.sort_values(by = ['PatientSerNum', 'date', 'FractionNumber'], inplace = True)
# data_part2_new.reset_index(drop = True, inplace = True)

In [66]:
# data_part1_new.to_csv(pat_duration_LSTM_data_path + 'data_part1_new.csv')
# data_part2_new.to_csv(pat_duration_LSTM_data_path + 'data_part2_new.csv')

# data_part1_new = pd.read_csv(pat_duration_LSTM_data_path + 'data_part1_new.csv')
# data_part2_new = pd.read_csv(pat_duration_LSTM_data_path + 'data_part2_new.csv')

In [10]:
data_part1_new.head()
print(f'The shape of data_part1_new is {data_part1_new.shape}')
print(f'The shape of data_part1_new is {data_part2_new.shape}')

DATA = pd.merge(data_part1_new, data_part2_new, on = ['PatientSerNum', 'date'], how = 'inner')
DATA.sort_values(by = ['PatientSerNum', 'AppointmentSerNum', 'ScheduledStartTime', 'FractionNumber'], inplace = True)
print(f'The shape of data is {DATA.shape}')

The shape of data_part1_new is (251892, 21)
The shape of data_part1_new is (142560, 13)
The shape of data is (106232, 32)


In [11]:
DATA_grouped = DATA.groupby('PatientSerNum')

series_count = DATA_grouped.count()
series_seq = series_count[series_count.AppointmentSerNum > 1].index.tolist()
series_one = series_count[series_count.AppointmentSerNum == 1].index.tolist()
print(f'series_seq {len(series_seq)}')
print(f'series_one {len(series_one)}')

series_seq = pd.DataFrame({'PatientSerNum': series_seq})
series_data = pd.merge(series_seq, DATA, on = 'PatientSerNum', how = 'inner')
series_data.sort_values(by = ['PatientSerNum', 'ScheduledStartTime'], inplace = True)

series_seq 6236
series_one 575


In [12]:
series_data_grouped = series_data.groupby('PatientSerNum')
pat_list = list(series_data_grouped.groups.keys())
APPT_LEN = series_data_grouped.count().AppointmentSerNum.max()
FEATURE_LEN = 438
print(f'The total number of patient is {len(pat_list)}')
print(f'The max length of patient is {APPT_LEN}')
print(f'The number of input encoded-feature is {FEATURE_LEN}')

The total number of patient is 6236
The max length of patient is 80
The number of input encoded-feature is 438


In [13]:
def generate_sample(train_sample, APPT_LEN, FEATURE_LEN):
    
    # 样本标签
    train_y = np.array([train_sample.Actual_duration.iloc[-1]])

    # 最后一个appointment 为我们需要预测的真实治疗时长所对应的appointment，所以需要设置为0
    train_sample.Actual_duration.iloc[-1] = 0

    # 因为存在相隔很远的两次预约，因此，构造特征Interval_scheduled 来度量两次预约之间的距离
    ## 上一次的预期治疗开始时间
    train_sample['Last_ScheduledStartTime'] = train_sample.ScheduledStartTime.shift(
        periods = 1, fill_value = train_sample.ScheduledStartTime.iloc[0])
    ## 需要将时间戳从字符型转为datetime 类型，从而计算时间间隔
    train_sample['Last_ScheduledStartTime'] = train_sample.Last_ScheduledStartTime.apply(lambda x: str_to_Datetime(x))
    train_sample['ScheduledStartTime'] = train_sample.ScheduledStartTime.apply(lambda x: str_to_Datetime(x))
    ## 时间间隔
    train_sample['Interval_scheduled'] = train_sample.apply(
        lambda x: (x.ScheduledStartTime - x.Last_ScheduledStartTime).days, axis = 1)
    
    # 对分类变量进行one-hot encoding处理
    encode_cate = pd.DataFrame({})
    
    # 这个地方将Sex 作为序列的一部分，并不是在最后的隐藏层进行合并
    encode_cate['Sex'] = train_sample['Sex'].apply(
        lambda x: sum(label_encoder_Sex.transform(np.array(x).reshape(-1,1))))

    encode_cate['dxt_AliasName'] = train_sample['dxt_AliasName'].apply(
        lambda x: sum(label_encoder_dxt_AliasName.transform(np.array(x).reshape(-1,1))))
    train_x = np.vstack(encode_cate.dxt_AliasName.tolist())

    encode_cate['AliasSerNum'] = train_sample['AliasSerNum'].apply(
        lambda x: sum(label_encoder_AliasSerNum.transform(np.array(x).reshape(-1,1))))
    train_x = np.hstack((train_x, np.vstack(encode_cate.AliasSerNum.tolist())))

    encode_cate['month'] = train_sample['month'].apply(
        lambda x: sum(label_encoder_month.transform(np.array(x).reshape(-1,1))))
    train_x = np.hstack((train_x, np.vstack(encode_cate.month.tolist())))

    encode_cate['week'] = train_sample['week'].apply(
        lambda x: sum(label_encoder_week.transform(np.array(x).reshape(-1,1))))
    train_x = np.hstack((train_x, np.vstack(encode_cate.week.tolist())))

    encode_cate['hour'] = train_sample['hour'].apply(
        lambda x: sum(label_encoder_hour.transform(np.array(x).reshape(-1,1))))
    train_x = np.hstack((train_x, np.vstack(encode_cate.hour.tolist())))

    encode_cate['DoctorSerNum'] = train_sample['DoctorSerNum'].apply(
        lambda x: sum(label_encoder_DoctorSerNum.transform(np.array(x).reshape(-1,1))))
    train_x = np.hstack((train_x, np.vstack(encode_cate.DoctorSerNum.tolist())))

    encode_cate['TreatmentOrientation'] = train_sample['TreatmentOrientation'].apply(
        lambda x: sum(label_encoder_TreatmentOrientation.transform(np.array(x).reshape(-1,1))))
    train_x = np.hstack((train_x, np.vstack(encode_cate.TreatmentOrientation.tolist())))

    encode_cate['FractionNumber'] = train_sample['FractionNumber'].apply(
        lambda x: sum(label_encoder_FractionNumber.transform(np.array(x).reshape(-1,1))))
    train_x = np.hstack((train_x, np.vstack(encode_cate.FractionNumber.tolist())))

    encode_cate['UserName'] = train_sample['UserName'].apply(
        lambda x: sum(label_encoder_UserName.transform(np.array(x).reshape(-1,1))))
    train_x = np.hstack((train_x, np.vstack(encode_cate.UserName.tolist())))

    encode_cate['CourseId'] = train_sample['CourseId'].apply(
        lambda x: sum(label_encoder_CourseId.transform(np.array(x).reshape(-1,1))))
    train_x = np.hstack((train_x, np.vstack(encode_cate.CourseId.tolist())))

    encode_cate['ResourceSerNum'] = train_sample['ResourceSerNum'].apply(
        lambda x: sum(label_encoder_ResourceSerNum.transform(np.array(x).reshape(-1,1))))
    train_x = np.hstack((train_x, np.vstack(encode_cate.ResourceSerNum.tolist())))
    
    # 将数值变量和非数值变量进行合并
    train_num = train_sample[feature_num]
    train_x = np.hstack((train_x, train_num))
    
#     # 需要满足序列长度的要求，因此对于短序列进行补零操作
#     zeros = np.zeros((APPT_LEN - train_x.shape[0], FEATURE_LEN))
#     train_x = np.vstack((zeros, train_x))
    
    return train_x, train_y

In [15]:
'Aladeeb Ems' in list(label_encoder_UserName.categories_[0])

False

In [22]:
'Aladeeb Ems' in list(data_part2.UserName.unique())

False

In [26]:
series_data.UserName.iloc[0]

"['vcaissie']"

In [23]:
series_data.UserName.unique()

array(["['vcaissie']", "['amahbub']", "['sfournie']", "['rbasdeo']",
       "['cguedes']", "['jleclerc']", "['kcrawley']", "['abelli']",
       "['swerbrou']", "['jlieu']", "['jstachura']", "['sleichen']",
       "['aseevara']", "['ltasillo']", "['sfournie', 'jstachura']",
       "['rkarant']", "['jbattista']", "['aseevara', 'rkarant']",
       "['mcaulfield']", "['epagter']", "['svilleneuve']", "['sgaglia']",
       "['adicorpo']", "['eliu']", "['vmarsillo']", "['midris']",
       "['rclark']", "['snowac']", "['agilcher']", "['mkarakova']",
       "['ijawandha']", "['lgoldenberg']",
       "['Boisvert-Huneault Christian']", "['Vachon Suzie']",
       "['nowac sydnee']", "['Aladeeb Emsal']", "['poon']",
       "['jawandha indudeep']", "['snelson']", "['jrabanal']",
       "['Unknown', 'jlieu']", "['sillick']", "['eli']", "['hli']",
       "['amendelsohn']", "['Unknown', 'ltasillo']", "['rislam']",
       "['Unknown', 'gdabizha']", "['Unknown', 'mpeuckert']", "['mlin']",
       "['Chen 

In [42]:
# 不指定时间戳长度
model_sequence = Sequential()
model_sequence.add(layers.LSTM(
        batch_input_shape = (None , None, FEATURE_LEN),
        output_dim = 128,
        dropout=0.1,
        recurrent_dropout=0.5,
        return_sequences=True,
        ))

model_sequence.add(layers.LSTM(
        output_dim = 32,
        dropout=0.1,
        recurrent_dropout=0.5,
        ))
# stateful = True 本次batch的参数返回到下一次的训练中

model_sequence.add(layers.Dense(1))

model_sequence.compile(
        optimizer = 'rmsprop',
        loss = 'mae'
        )

model_sequence.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_7 (LSTM)                (None, None, 128)         290304    
_________________________________________________________________
lstm_8 (LSTM)                (None, 32)                20608     
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 33        
Total params: 310,945
Trainable params: 310,945
Non-trainable params: 0
_________________________________________________________________


In [17]:
BATCH_train = 40
BATCH_test = 5

In [18]:
for i in range(int(len(pat_list)/(BATCH_train + BATCH_test))):
    print(f'Batch {i}')
    # 构造训练数据
    train_x = []
    train_y = []
    for tr in range((BATCH_train+BATCH_test)*i,
                    (BATCH_train+BATCH_test)*(i+1) - BATCH_test):
        pat = pat_list[tr]
        train_sample = series_data_grouped.get_group(pat)
        train_x_i, train_y_i = generate_sample(train_sample, APPT_LEN, FEATURE_LEN)
        train_x.append(train_x_i)
        train_y.append(train_y_i)
    train_x = np.array(train_x)
    train_y = np.array(train_y)
    train_result = model_sequence.train_on_batch(train_x, train_y)
        
    val_x = []
    val_y = []
    for te in range((BATCH_train+BATCH_test)*(i+1) - BATCH_test,
                    (BATCH_train+BATCH_test)*(i+1)):
        pat = pat_list[tr]
        train_sample = series_data_grouped.get_group(pat)
        val_x_i, val_y_i = generate_sample(train_sample, APPT_LEN, FEATURE_LEN)
        val_x.append(val_x_i)
        val_y.append(val_y_i)
    val_x = np.array(val_x)
    val_y = np.array(val_y)
    break
    # model_sequence.train_on_batch(val_x, val_y)
    val_result = model_sequence.evaluate(val_x, val_y, verbose=0) # verbose 显示的时候有进度条
    print(f'{tr+1 - BATCH_train}-{tr+1}-train [{train_result}] ||| {te+1 - BATCH_test}-{te+1}-validation [{val_result}]')

Batch 0


ValueError: y contains previously unseen labels: ['Aladeeb Emsa']

In [27]:
# dropout=0.1,
# recurrent_dropout=0.5,

model_sequence1 = Sequential()
model_sequence1.add(layers.GRU(
        batch_input_shape = (None , APPT_LEN, FEATURE_LEN),
        output_dim = 128,

        return_sequences=True,
        ))

# dropout=0.1,
# recurrent_dropout=0.5,

model_sequence1.add(layers.GRU(
        output_dim = 32,

        ))
# stateful = True 本次batch的参数返回到下一次的训练中

model_sequence1.add(layers.Dense(1))

model_sequence1.compile(
        optimizer = 'rmsprop',
        loss = 'mae'
        )

model_sequence1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_5 (GRU)                  (None, 80, 128)           217728    
_________________________________________________________________
gru_6 (GRU)                  (None, 32)                15456     
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 33        
Total params: 233,217
Trainable params: 233,217
Non-trainable params: 0
_________________________________________________________________


In [28]:
import warnings
warnings.filterwarnings("ignore")


for i in range(int(len(pat_list)/(BATCH_train + BATCH_test))):
    print(f'Batch {i}')
    # 构造训练数据
    train_x = []
    train_y = []
    for tr in range((BATCH_train+BATCH_test)*i,
                    (BATCH_train+BATCH_test)*(i+1) - BATCH_test):
        pat = pat_list[tr]
        train_sample = series_data_grouped.get_group(pat)
        train_x_i, train_y_i = generate_sample(train_sample, APPT_LEN, FEATURE_LEN)
        train_x.append(train_x_i)
        train_y.append(train_y_i)
    train_x = np.array(train_x)
    train_y = np.array(train_y)
    train_result = model_sequence1.train_on_batch(train_x, train_y)
        
    val_x = []
    val_y = []
    for te in range((BATCH_train+BATCH_test)*(i+1) - BATCH_test,
                    (BATCH_train+BATCH_test)*(i+1)):
        pat = pat_list[tr]
        train_sample = series_data_grouped.get_group(pat)
        val_x_i, val_y_i = generate_sample(train_sample, APPT_LEN, FEATURE_LEN)
        val_x.append(val_x_i)
        val_y.append(val_y_i)
    val_x = np.array(val_x)
    val_y = np.array(val_y)
    # model_sequence.train_on_batch(val_x, val_y)
    val_result = model_sequence1.evaluate(val_x, val_y, verbose=0) # verbose 显示的时候有进度条
    print(f'{tr+1 - BATCH_train}-{tr+1}-train [{train_result}] ||| {te+1 - BATCH_test}-{te+1}-validation [{val_result}]')

Batch 0
0-40-train [29.033550262451172] ||| 40-45-validation [17.26938247680664]
Batch 1
45-85-train [18.42552375793457] ||| 85-90-validation [15.728889465332031]
Batch 2
90-130-train [17.172286987304688] ||| 130-135-validation [10.300146102905273]
Batch 3
135-175-train [20.477676391601562] ||| 175-180-validation [20.931716918945312]
Batch 4
180-220-train [15.885931015014648] ||| 220-225-validation [14.639741897583008]
Batch 5
225-265-train [19.633190155029297] ||| 265-270-validation [7.4754228591918945]
Batch 6
270-310-train [13.881879806518555] ||| 310-315-validation [20.360618591308594]
Batch 7
315-355-train [14.46210765838623] ||| 355-360-validation [3.0205907821655273]
Batch 8
360-400-train [12.831311225891113] ||| 400-405-validation [14.861434936523438]
Batch 9
405-445-train [14.254976272583008] ||| 445-450-validation [5.8323445320129395]
Batch 10
450-490-train [12.768884658813477] ||| 490-495-validation [6.7624006271362305]
Batch 11
495-535-train [12.343925476074219] ||| 535-540

KeyboardInterrupt: 